<img src="http://imgur.com/1ZcRyrc.png" style="float: left; margin: 20px; height: 55px">

# `GridSearch` with `keras`

_Authors: Riley Dallas (ATX)_

---


### Learning Objectives

- Learn how to use `keras`'s `scikit_learn` wrappers.
- Use `GridSearchCV` to tune a `keras` model

In [1]:
from sklearn.datasets import make_regression
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn import metrics

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.wrappers.scikit_learn import KerasRegressor

# Regression
---

`make_regression` is an excellent library for generating a random regression problem. This helps us focus on the model without having to worry about the dataset. 

In the cell below, use `make_regression` to generate 10,000 samples using 20 features.

In [2]:
X, y = make_regression(n_samples = 10_000, n_features = 20, random_state = 33)

### Train/Test Split
---

We always want to have a validation set to test our model. Use the `train_test_split` function to split our `X` and `y` variables into a training set and a holdout set.

In [3]:
X_train, X_test, y_train, y_test = train_test_split(X, y,
                                               random_state = 33)

In [4]:
X_train

array([[ 1.41189076, -0.91347111,  0.14259087, ...,  0.12344782,
        -0.6496142 ,  1.52096123],
       [-1.02816164,  0.98753556, -0.6275951 , ..., -0.48457181,
         0.17234647,  2.44895862],
       [-0.80786716, -0.30427438,  1.37881299, ...,  0.59347582,
         2.23387656, -0.71765433],
       ...,
       [-1.63249941,  0.0194359 , -1.33280575, ..., -0.19320821,
        -0.03490674, -1.75731354],
       [-0.36617357, -0.99893844, -0.19514583, ..., -0.31899267,
         0.85873147, -1.29094535],
       [-0.58679975, -0.50215483, -0.35894277, ...,  0.39512846,
         0.85427539,  0.13127747]])

In [5]:
X_train.shape

(7500, 20)

### `StandardScaler`
---

You want to scale your data for *any* model that uses Gradient Descent, which includes Neural Networks.

In [6]:
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

In [7]:
X_train[:5]
#data set has been centered and scaled

array([[ 1.42655114e+00, -9.11952879e-01,  1.38036003e-01,
        -2.01847908e+00, -1.47087576e+00, -5.81915264e-01,
        -8.04751724e-01,  1.37427522e+00,  1.11047204e+00,
         2.05078273e+00, -7.53627555e-01, -8.64710847e-01,
        -4.14116197e-01, -1.79552709e-01, -7.95286428e-01,
         1.87279362e-03,  6.49213492e-01,  1.23958264e-01,
        -6.54504704e-01,  1.54373286e+00],
       [-1.01397260e+00,  9.83207069e-01, -6.33044824e-01,
         6.08844786e-01, -1.87112879e+00, -2.88773446e-01,
         7.75784925e-01,  1.88356581e-01, -1.34772813e+00,
        -1.38735230e-01,  7.82067489e-01,  7.26554639e-01,
        -2.04091906e-01,  1.84734529e-01, -4.31466164e-01,
        -8.67254109e-01, -6.70581820e-01, -4.86615351e-01,
         1.60283694e-01,  2.47276938e+00],
       [-7.93635569e-01, -3.04629791e-01,  1.37569445e+00,
         3.30225661e-01,  5.72573035e-01,  1.09430600e-01,
         3.47545543e-02, -5.53626571e-01,  5.82888914e-01,
        -1.46615089e+00, -3.3

### Create your network topology
---

We'll create a neural network like we've done before, only this time we'll wrap the entire model in a function. Once we have that set up, we can use the `KerasRegressor` wrapper to set it up as an `sklearn` model, which we'll then apply `GridSearchCV`.

In [8]:
def model_func(layer_one_neurons = 32, layer_two_neurons = 32):
    #instantiate the model:
    model = Sequential()
    
    #set up first layer:
    model.add(Dense(layer_one_neurons, activation = 'relu',
                    input_shape = (20, )))
    
    #set up second layer:
    model.add(Dense(layer_two_neurons, activation = 'relu'))
    
    #add output layer:
    model.add(Dense(1, activation = None))
    
    model.compile(loss = 'mse', optimizer = 'adam')
    
    return model

In [9]:
nn = KerasRegressor(build_fn = model_func, batch_size = 512, verbose = 0)

<ipython-input-9-4b1f657b3015>:1: DeprecationWarning: KerasRegressor is deprecated, use Sci-Keras (https://github.com/adriangb/scikeras) instead.
  nn = KerasRegressor(build_fn = model_func, batch_size = 512, verbose = 0)


In [11]:
param_grid = {
    'epochs' : [1000, 100],
    'layer_one_neurons' : [32, 42],
    'layer_two_neurons' : [32, 42],
    
}

#gridsearch:

gs = GridSearchCV(nn, param_grid = param_grid, cv = 3)

In [12]:
gs.fit(X_train, y_train)
print(gs.best_score_)
gs.best_params_

-0.034317836786309876


{'epochs': 1000, 'layer_one_neurons': 32, 'layer_two_neurons': 42}

In [13]:
preds = gs.predict(X_test)

In [14]:
metrics.r2_score(y_test, preds)

0.9999998974715377

In [15]:
preds_tr = gs.predict(X_train)

In [16]:
metrics.r2_score(y_train, preds_tr)

0.9999999999227697

In [17]:
#lets create a large gridsearch - adding regularization

In [ ]:
def model_fn_deep(hidden_neurons = 32, hidden_layers = 5, dropout = 0.5):
    model = Sequential()
    for layer in range(hidden_layers):
        if layer == 0 :
            model.add(Dense(hidden_neurons, activation = 'relu',
                            input_shape = (20, )))
            model.add(Dropout(dropout))
        
        else:
            model.add(Dense(hidden_neurons, activation = 'relu'))
            model.add(Dropout(dropout))
    model.add(Dense(1, activation = None))
    
    model.compile(loss = 'mse', optimizer = 'adam')
    
    return model

In [19]:
nn = KerasRegressor(build_fn = model_fn_deep, batch_size = 512, verbose = 0)

<ipython-input-19-bf8374d2d9ca>:1: DeprecationWarning: KerasRegressor is deprecated, use Sci-Keras (https://github.com/adriangb/scikeras) instead.
  nn = KerasRegressor(build_fn = model_fn_deep, batch_size = 512, verbose = 0)


In [20]:
nn_param_grid = {
    'epochs' : [10, 20],
    'hidden_neurons' : [20, 30],
    'hidden_layers' : [3, 5, 7],
    'dropout' : [.1, .2, .5]
}

In [21]:
gs_deep = GridSearchCV(nn, param_grid = nn_param_grid, cv = 5)

In [22]:
gs_deep.fit(X_train, y_train)

GridSearchCV(cv=5,
             estimator=<keras.wrappers.scikit_learn.KerasRegressor object at 0x0000028909D86D90>,
             param_grid={'dropout': [0.1, 0.2, 0.5], 'epochs': [10, 20],
                         'hidden_layers': [3, 5, 7],
                         'hidden_neurons': [20, 30]})

In [23]:
print(gs_deep.best_score_)
gs_deep.best_params_

-637.0611328125


{'dropout': 0.1, 'epochs': 20, 'hidden_layers': 7, 'hidden_neurons': 30}

In [24]:
train_preds = gs_deep.predict(X_train)
test_preds = gs_deep.predict(X_test)

In [25]:
print(metrics.r2_score(y_train, train_preds))

0.9883597644899899


In [26]:
print(metrics.r2_score(y_test, test_preds))

0.9879561752349176


In [27]:
from sklearn.linear_model import LinearRegression

In [28]:
linreg = LinearRegression()

In [29]:
linreg.fit(X_train, y_train)

LinearRegression()

In [30]:
linreg.score(X_train, y_train)

1.0